In [ ]:
# Отобразите все записи из таблицы company по компаниям, которые закрылись.
SELECT *
FROM company
WHERE status = 'closed'

In [ ]:
# Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле funding_total.
SELECT funding_total
FROM company
WHERE category_code = 'news'
  AND country_code = 'USA'
ORDER BY funding_total DESC;

In [ ]:
# Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
  AND acquired_at BETWEEN '2011-01-01' AND '2013-12-31';

In [ ]:
# Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на 'Silver'.
SELECT
    first_name,
    last_name,
    network_username
FROM people
WHERE network_username LIKE 'Silver%'


In [ ]:
# Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку 'money', а фамилия начинается на 'K'.
SELECT *
FROM people
WHERE network_username LIKE '%money%'
    AND last_name LIKE 'K%'


In [ ]:
# Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.
SELECT
  country_code,
  SUM(COALESCE(funding_total, 0)) AS total_funding
FROM company
GROUP BY country_code
ORDER BY total_funding DESC;

In [ ]:
# Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
# Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.
SELECT funded_at,
    MIN(raised_amount) AS min_investment,
    MAX(raised_amount) AS max_investment
FROM funding_round
WHERE funded_at IS NOT NULL
GROUP BY funded_at
HAVING 
    MIN(raised_amount) != 0
    AND MIN(raised_amount) <> MAX(raised_amount);

In [ ]:
# Создайте поле с категориями:
# Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию high_activity.
# Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию middle_activity.
# Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию low_activity.
# Отобразите все поля таблицы fund и новое поле с категориями.
SELECT *,
    CASE
        WHEN invested_companies >= 100 THEN 'high_activity'
        WHEN invested_companies >= 20 THEN 'middle_activity'
        ELSE 'low_activity'
    END AS activity_category
FROM fund;


In [ ]:
# Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонды принимали участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.
SELECT
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
       ROUND(AVG(investment_rounds)) AS avg_rounds
FROM fund
GROUP BY activity
ORDER BY avg_rounds;

In [ ]:
# Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 
# Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю. 
# Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.
SELECT
    country_code,
    MIN(invested_companies) AS min_invested,
    MAX(invested_companies) AS max_invested,
    AVG(invested_companies) AS avg_invested
FROM fund
WHERE founded_at BETWEEN '2010-01-01' AND '2012-12-31'
    AND country_code IS NOT NULL
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY avg_invested DESC, country_code ASC
LIMIT 10;

In [ ]:
# Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.
SELECT
  p.first_name,
  p.last_name,
  e.instituition
FROM people AS p
LEFT JOIN education AS e ON p.id = e.person_id;

In [ ]:
# Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.
SELECT
    c.name AS company_name,
    COUNT(DISTINCT e.instituition) AS unique_universities
FROM people AS p
JOIN education AS e ON p.id = e.person_id
JOIN company AS c ON p.company_id = c.id
GROUP BY c.name 
ORDER BY unique_universities DESC
LIMIT 5;

In [ ]:
# Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.
SELECT DISTINCT c.name
FROM company AS c
JOIN funding_round AS f ON c.id = f.company_id
WHERE c.status = 'closed'
  AND f.is_first_round = 1
  AND f.is_last_round = 1;

In [ ]:
# Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.
SELECT DISTINCT p.id
FROM people AS p
JOIN company AS c ON p.company_id = c.id
JOIN funding_round AS f ON c.id = f.company_id
WHERE c.status = 'closed'
  AND f.is_first_round = 1
  AND f.is_last_round = 1;

In [ ]:
# Посчитайте количество учебных заведений для каждого сотрудника. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.
SELECT p.id AS person_id, COUNT(e.instituition) AS institution_count
FROM people AS p
JOIN education AS e ON p.id = e.person_id
WHERE p.company_id IN (
    SELECT f.company_id
    FROM funding_round AS f
    JOIN company AS c ON f.company_id = c.id
    WHERE c.status = 'closed'
      AND f.is_first_round = 1
      AND f.is_last_round = 1
)
GROUP BY p.id;

In [ ]:
# Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.
SELECT AVG(edu_count) AS avg_institution_count
FROM (
    SELECT p.id AS person_id, COUNT(e.instituition) AS edu_count
    FROM people AS p
    JOIN education AS e ON p.id = e.person_id
    WHERE p.company_id IN (
        SELECT f.company_id
        FROM funding_round AS f
        JOIN company AS c ON f.company_id = c.id
        WHERE c.status = 'closed'
          AND f.is_first_round = 1
          AND f.is_last_round = 1
    )
    GROUP BY p.id
) AS subquery;


In [ ]:
# Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Socialnet.
SELECT AVG(edu_count) AS avg_institution_count
FROM (
    SELECT p.id AS person_id, COUNT(e.instituition) AS edu_count
    FROM people AS p
    JOIN education AS e ON p.id = e.person_id
    JOIN company AS c ON p.company_id = c.id
    WHERE c.name = 'Socialnet'
    GROUP BY p.id
) AS subquery;

In [ ]:
# Составьте таблицу из полей:
# name_of_fund — название фонда;
# name_of_company — название компании;
# amount — сумма инвестиций, которую привлекла компания в раунде.
# В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.
SELECT 
    f.name AS name_of_fund,
    c.name AS name_of_company,
    fr.raised_amount AS amount
FROM investment AS i
JOIN fund AS f ON i.fund_id = f.id
JOIN funding_round AS fr ON i.funding_round_id = fr.id
JOIN company AS c ON fr.company_id = c.id
WHERE c.milestones > 6
  AND fr.funded_at BETWEEN '2012-01-01' AND '2013-12-31';

In [ ]:
# Выгрузите таблицу, в которой будут такие поля:
# название компании-покупателя;
# сумма сделки;
# название компании, которую купили;
# сумма инвестиций, вложенных в купленную компанию;
# доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.
# Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
# Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.
SELECT 
    buyer.name AS acquiring_company,
    a.price_amount,
    target.name AS acquired_company,
    target.funding_total,
    ROUND(a.price_amount / target.funding_total) AS multiple
FROM acquisition AS a
JOIN company AS buyer ON a.acquiring_company_id = buyer.id
JOIN company AS target ON a.acquired_company_id = target.id
WHERE a.price_amount > 0
  AND target.funding_total > 0
ORDER BY a.price_amount DESC, target.name ASC
LIMIT 10;

In [ ]:
# Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.
SELECT company.name,
EXTRACT(MONTH FROM funding_round.funded_at) AS month
FROM company
JOIN funding_round ON funding_round.company_id = company.id
WHERE company.category_code = 'social'
AND funding_round.raised_amount > 0
AND funding_round.funded_at >= '2010-01-01'
AND funding_round.funded_at < '2014-01-01'
ORDER BY company.name, month;

In [ ]:
# Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:
# номер месяца, в котором проходили раунды;
# количество уникальных названий фондов из США, которые инвестировали в этом месяце;
# количество компаний, купленных за этот месяц;
# общая сумма сделок по покупкам в этом месяце.
WITH investment_data AS (
    -- Инвестиции от фондов из США, с датой в периоде 2010–2013 гг.
    SELECT 
        EXTRACT(MONTH FROM f.funded_at) AS month_number,
        i.fund_id
    FROM funding_round f
    JOIN investment i ON f.id = i.funding_round_id
    JOIN fund fd ON i.fund_id = fd.id
    WHERE EXTRACT(YEAR FROM f.funded_at) BETWEEN 2010 AND 2013
      AND fd.country_code = 'USA'
),

unique_funds_per_month AS (
    -- Уникальные фонды из США по месяцам
    SELECT 
        month_number,
        COUNT(DISTINCT fund_id) AS us_funds_count
    FROM investment_data
    GROUP BY month_number
),

acquisition_data AS (
    -- Данные о покупках компаний в тот же период
    SELECT 
        EXTRACT(MONTH FROM acquired_at) AS month_number,
        COUNT(*) AS companies_acquired,
        SUM(price_amount) AS total_deal_value
    FROM acquisition
    WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
    GROUP BY EXTRACT(MONTH FROM acquired_at)
)

-- Соединяем оба результата по номеру месяца
SELECT 
    COALESCE(u.month_number, a.month_number) AS month_number,
    COALESCE(u.us_funds_count, 0) AS us_funds_count,
    COALESCE(a.companies_acquired, 0) AS companies_acquired,
    COALESCE(a.total_deal_value, 0) AS total_deal_value
FROM unique_funds_per_month u
FULL OUTER JOIN acquisition_data a
ON u.month_number = a.month_number
ORDER BY month_number;

In [ ]:
# Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.
WITH
     inv_2011 AS (SELECT country_code,
                         AVG(funding_total) AS avg_2011
                  FROM company
                  WHERE EXTRACT(YEAR FROM founded_at) = 2011
                  GROUP BY country_code),
     inv_2012 AS (SELECT country_code,
                         AVG(funding_total) AS avg_2012
                  FROM company
                  WHERE EXTRACT(YEAR FROM founded_at) = 2012
                  GROUP BY country_code),
     inv_2013 AS (SELECT country_code,
                         AVG(funding_total) AS avg_2013
                  FROM company
                  WHERE EXTRACT(YEAR FROM founded_at) = 2013
                  GROUP BY country_code)
SELECT inv_2011.country_code,
       inv_2011.avg_2011,
       inv_2012.avg_2012,
       inv_2013.avg_2013
FROM inv_2011
INNER JOIN inv_2012 ON inv_2011.country_code = inv_2012.country_code
INNER JOIN inv_2013 ON inv_2013.country_code = inv_2012.country_code
ORDER BY inv_2011.avg_2011 DESC;